In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.svm import LinearSVC
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [17]:
def dataPreProcess():
    data = pd.read_csv('yelp.csv')
    yelpReviews = data[(data['stars'] == 1) | (data['stars'] == 5)]
    X = yelpReviews['text']
    y = yelpReviews['stars']
    return X,y

In [18]:
def SVM(X,Y):
    tokens=[]
    for review in X:
        tokens.append(review.split())

    binarizer = MultiLabelBinarizer()
    binarizer.fit(tokens)

    X_train, X_test, y_train, y_test = train_test_split(tokens,Y ,test_size=0.25, random_state=111)

    lsvm = LinearSVC()
    lsvm.fit(binarizer.transform(X_train), y_train)
    score = lsvm.score(binarizer.transform(X_test), y_test)

    print(score)
    print('\n')
    print(classification_report(y_test, lsvm.predict(binarizer.transform(X_test))))

In [19]:
X2,y=dataPreProcess()
X=X2.tolist()
SVM(X,y)

0.9148727984344422


             precision    recall  f1-score   support

          1       0.86      0.68      0.76       202
          5       0.93      0.97      0.95       820

avg / total       0.91      0.91      0.91      1022



In [20]:
X,y=dataPreProcess()

## Feature Selection

In [6]:
Vectorizer = CountVectorizer(analyzer='char',ngram_range=(2, 2)).fit(X)

In [142]:
Vectorizer = CountVectorizer(analyzer='char',ngram_range=(2, 2), max_df=0.7 ).fit(X)

In [25]:
Vectorizer = CountVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.7).fit(X)

In [151]:
Vectorizer = CountVectorizer(analyzer='char',ngram_range=(3, 3)).fit(X)

In [115]:
Vectorizer = CountVectorizer(analyzer='word',stop_words='english').fit(X)

In [11]:
Vectorizer = CountVectorizer(analyzer='word').fit(X) #simple

In [21]:
Vectorizer = CountVectorizer(analyzer='word',stop_words='english',max_features=9000).fit(X)

## Transformation 

In [22]:
X = Vectorizer.transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

## Multinomial NaivesBaeys

In [23]:
nb = MultinomialNB()
nb.fit(X_train, y_train)
preds = nb.predict(X_test)

In [24]:
print(confusion_matrix(y_test, preds))
print('\n')
print(classification_report(y_test, preds))

[[186  42]
 [ 47 951]]


             precision    recall  f1-score   support

          1       0.80      0.82      0.81       228
          5       0.96      0.95      0.96       998

avg / total       0.93      0.93      0.93      1226



## VaderAnalysis

In [15]:
def getVaderAnalysis(text):
    snt = analyzer.polarity_scores(text)
    if(snt['pos']>snt['neg']):
        return 5
    else:
        return 1

In [16]:
analyzer = SentimentIntensityAnalyzer()
results=[]
for reviews in X2:
    results.append(getVaderAnalysis(reviews))
print(classification_report(y, results))

             precision    recall  f1-score   support

          1       0.83      0.56      0.67       749
          5       0.91      0.97      0.94      3337

avg / total       0.89      0.90      0.89      4086

